# Chroma Default Embedding demo

In [1]:
# https://ollama.com/blog/embedding-models

import chromadb
from rich import print  # https://rich.readthedocs.io/en/stable/markup.html#console-markup

In [2]:
documents = [
    "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
    "Llamas were first domesticated and used as pack animals 4,001 to 5,001 years ago in the Peruvian highlands",
    "Llamas can grow as much as 8 feet tall though the average llama between 7 feet 8 inches and 7 feet 11 inches tall",
    "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
    "Llamas are vegetarians and have very efficient digestive systems",
    "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
    "Many places in South America, including Peru, have packs of llamas in the mountains.",
    "Dogs have an average life-span of a dozen years or so.",
    "House cats generally do not live in packs, they are often solitary animals, even in Cusco.",
    "Dogs are not good pack animals, but do often live in packs, sometimes in Lima",
    "Mules are beasts of burden; they can carry substantial loads on trips."
]

questions = {
    1:{'q':"How much do llamas weight?", 'ids':[3,2]},
    2:{'q':'Where were llamas domesticated?', 'ids':[1,6]},
    3:{'q':'llamas domesticated', 'ids':[1,6]},
    4:{'q':'What animal is in Peru?', 'ids':[1,6]},
    5:{'q':'good pack animal?', 'ids':[1,10]},
    6:{'q':'animals live in packs', 'ids':[9,8]},
    7:{'q':'How long do animals live?', 'ids':[7,5]},
}

In [3]:
client = chromadb.Client()

collections = client.list_collections()
print(f"Collections: {collections}")

col_name = 'test-collection'
if(col_name in collections):
    collection = client.get_collection(name=col_name) # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
    print(f"[green]Found collection: {collection}[/]")
else:
    print(f"[blue]Could not find existing collection: {col_name}, create it now...[/]")
    collection = client.create_collection(name=col_name)

Collections: []

Could not find existing collection: test-collection, create it now...

In [4]:
collection

Collection(name=test-collection)

In [5]:
# store each document in a vector embedding database
for i, d in enumerate(documents):
    print(f"{i}) add doc: {d}")
    collection.add(
        ids=[str(i)],
        documents=[d]
    )

0) add doc: Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels

1) add doc: Llamas were first domesticated and used as pack animals 4,001 to 5,001 years ago in the Peruvian 
highlands

2) add doc: Llamas can grow as much as 8 feet tall though the average llama between 7 feet 8 inches and 7 feet 11 
inches tall

3) add doc: Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight

4) add doc: Llamas are vegetarians and have very efficient digestive systems

...

6) add doc: Many places in South America, including Peru, have packs of llamas in the mountains.

7) add doc: Dogs have an average life-span of a dozen years or so.

8) add doc: House cats generally do not live in packs, they are often solitary animals, even in Cusco.

9) add doc: Dogs are not good pack animals, but do often live in packs, sometimes in Lima

10) add doc: Mules are beasts of burden; they can carry substantial loads on trips.

In [6]:
existing_count = collection.count()
print(f"existing doc count: {existing_count}")
ids = collection.get(include=[])
print(f"collection ids: {ids}")

existing doc count: 11

collection ids: {'ids': ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], 'embeddings': None, 'metadatas': 
None, 'documents': None, 'uris': None, 'data': None}

In [27]:
# note: if tweaking table output, you likely want to re-run this cell along with the next, otherwise `rich` just appends to the existing table
from rich.table import Table
from rich.console import Console

console = Console()
table = Table(title="Simple Semantic Search Results", show_lines=True)
table.add_column("#" )  # no_wrap=True
table.add_column("Status" )  # no_wrap=True
table.add_column("Content")  # no_wrap=True
table.add_column("Score" )

style_good = 'green'
style_failed = 'red bold'
style_info = 'bright_black'

In [28]:
# print(f"\nready for queries on collection: {col_name}")
for q_number in questions:
    prompt = questions[q_number]['q']
    expected_ids = questions[q_number]['ids']
    table.add_row(f"{q_number})",  f"QUESTION", f"{prompt}", style='navy_blue on grey84 bold')
    # use default collection embedding function for the prompt and retrieve the most relevant doc
    results = collection.query(
        query_texts=[prompt],
        n_results=3
    )

    answers = results["documents"][0]
    for i, answer in enumerate(answers):
        id = int(results["ids"][0][i])
        distance = results["distances"][0][i]
        if(i < len(expected_ids)):
            expected = expected_ids[i]
            if(id==expected):
                # print(f"\t[green bold]got({id}):expected({expected}) (distance:{distance:.2f}): {answer}[/]")
                table.add_row('', f"[{style_good}]got({id}) \nexpected({expected})[/]", f"[{style_good}]{answer}[/]",  f"[{style_good}]{distance:.2f}")
            else:
                table.add_row('', f"[{style_failed}]got({id})[/] \n[{style_info}]expected({expected})[/]", f"[{style_failed}]{answer}[/] \n[{style_info}]{documents[expected]}[/]" ,  f"[{style_failed}]{distance:.2f}")
        else:
            # table.add_row('', f"[light_slate_grey](outside test)[/]", f"[light_slate_grey]{answer}[/]", f"{distance:.2f}")
            table.add_row('', '', f"[{style_info}]{answer}[/]", f"{distance:.2f}")

console.print(table)

                                          Simple Semantic Search Results                                           
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ #  ┃ Status       ┃ Content                                                                             ┃ Score ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ 1) │ QUESTION     │ How much do llamas weight?                                                          │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(3)       │ Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their     │ 0.24  │
│    │ expected(3)  │ body weight                                                                         │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(2)       │ Llamas can grow as much as 8 feet tall though the average llama between 7 feet 8    │ 0.72  │
│    │ expected(2)  │ inches and 7 feet 11 inches tall                                                    │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │              │ Llamas are vegetarians and have very efficient digestive systems                    │ 0.75  │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ 2) │ QUESTION     │ Where were llamas domesticated?                                                     │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(1)       │ Llamas were first domesticated and used as pack animals 4,001 to 5,001 years ago in │ 0.48  │
│    │ expected(1)  │ the Peruvian highlands                                                              │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(5)       │ Llamas live to be about 20 years old, though some only live for 15 years and others │ 0.71  │
│    │ expected(6)  │ live to be 30 years old                                                             │       │
│    │              │ Many places in South America, including Peru, have packs of llamas in the           │       │
│    │              │ mountains.                                                                          │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │              │ Llamas are vegetarians and have very efficient digestive systems                    │ 0.76  │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ 3) │ QUESTION     │ llamas domesticated                                                                 │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(1)       │ Llamas were first domesticated and used as pack animals 4,001 to 5,001 years ago in │ 0.59  │
│    │ expected(1)  │ the Peruvian highlands                                                              │       │
├────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────┤
│    │ got(4)       │ Llamas are vegetarians and have very efficient digestive systems                    │ 0.66  │
│    │ expected(6)  │ Many places in South America, including Peru, have packs of llamas in the           │       │
│    │              │ mountains.                                                                          │       │
├────┼──────────────┼───────────────────────────────────